In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive.flush_and_unmount()


# **Cycles/Eevee render**

In [ ]:
# # # # # # #Connect Google Drivebxhjxjxh
from google.colab import drive
drive.mount('/content/drive')

# Download Blender from Repository - Run Once unless version changed
# # # # # # # # #install Blender
!wget https://download.blender.org/release/Blender4.4/blender-4.4.0-linux-x64.tar.xz
!tar xf blender-4.4.0-linux-x64.tar.xz



MAIL = 'example@GMAIL.COM'

# # # # #set the paths to Blender Files
filename = f'/content/drive/MyDrive/BLENDER/{MAIL}/BLACKOUT/BLACKOUT_4.4.blend'
OUTPUT = f'/content/drive/MyDrive/BLENDER/{MAIL}/BLACKOUT/RESULT/####'

!./blender-4.4.0-linux-x64/blender -b $filename -noaudio -E 'CYCLES' -S 'Scene' -o $OUTPUT   -s 100 -e 200 -a -F 'PNG' -- --cycles-device CUDA


In [ ]:
# Set the paths to Blender Files in Python
import os

MAIL = 'example@GMAIL.COM'
filename = f'/content/drive/MyDrive/BLENDER/{MAIL}/SPEED/SPEED4.blend'
output = f'/content/drive/MyDrive/BLENDER/{MAIL}/SPEED/RESULT5.3/####'

# Export variables to the environment
os.environ['filename'] = filename
os.environ['output'] = output

# Run the rendering process using Bash
!if [ ! -f "$filename" ]; then \
    echo "Error: Blender file not found at '$filename'"; \
    exit 1; \
fi

!for frame in $(seq 181 220); do \
    if [ ! -f "$filename" ]; then \
        echo "Error: Blender file not found during rendering at frame $frame."; \
        exit 1; \
    fi; \
    echo "Rendering frame $frame started..."; \
    ./blender-4.3.0-linux-x64/blender -b "$filename" -noaudio -E 'CYCLES' -S 'Scene.003' -o "$output" -s "$frame" -e "$frame" -a -F 'PNG' -- --cycles-device CUDA > /dev/null 2>&1; \
    echo "Rendering frame $frame completed and saved."; \
done


Rendering frame 181 started...
Rendering frame 181 completed and saved.
Rendering frame 182 started...
Rendering frame 182 completed and saved.
Rendering frame 183 started...
Rendering frame 183 completed and saved.
Rendering frame 184 started...


In [ ]:
#workbench
!./blender-4.3.0-linux-x64/blender -b $filename -noaudio -E 'BLENDER_WORKBENCH' -o $OUTPUT -s 1 -e 400 -a -F 'FFMPEG' -- --cycles-device CUDA


In [ ]:
#CPU CYCLES
# !DISPLAY=:0 ./blender-3.3.8-linux-x64/blender -b $filename -noaudio -E 'BLENDER_WORKBENCH' -o $OUTPUT -s 200 -e 200 -a -F 'FFMPEG'

!./blender-3.3.8-linux-x64/blender -b $filename -noaudio -E 'CYCLES' -o $OUTPUT  -s 1 -e 50 -a -F 'PNG'


Blender 3.3.8 (hash 6a4c537c211e built 2023-06-20 00:26:39)
Read blend: /content/drive/MyDrive/desertride/RIDE.2.blend
WARN (bke.anim_sys): source/blender/blenkernel/intern/anim_sys.c:4240 BKE_animsys_eval_driver: invalid driver - ["prop"][0]
find_node_operation: Failed for (RIGIDBODY_REBUILD, '')
--------------------------------------------------------------------
Failed to add relation "Rigid Body Rebuild -> Point Cache Reset"
Could not find op_from: OperationKey(type: TRANSFORM, component name: '', operation code: RIGIDBODY_REBUILD)

Trace:

Depth    Type                     Name
-----    ----                     ----
1        Object                   Camera.001
2        Object                   chassi.018_chassi.10_0.001
3        Object                   Cube.001

Fra:1 Mem:277.10M (Peak 314.30M) | Time:00:01.89 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | door_lf_ok.013_chassi.10_0.001
Fra:1 Mem:277.71M (Peak 314.30M) | Time:00:01.89 | Mem:0.00M, Peak:0.00M 

In [ ]:
import os
import re

# Path to the folder containing the frames
folder_path = f'/content/drive/MyDrive/BLENDER/{MAIL}/BLACKOUT/RESULT/'

# List all files in the folder
files = os.listdir(folder_path)

# Extract frame numbers using a regular expression
frame_numbers = []
for file in files:
    match = re.match(r"(\d+)", file)  # Match digits at the start of the filename
    if match:
        frame_numbers.append(int(match.group(1)))

# Sort the frame numbers
frame_numbers.sort()

# Check for missing frames (assuming frames from 1 to 700)
missing_frames = [i for i in range(1, 701) if i not in frame_numbers]

if missing_frames:
    print("Missing frames:", missing_frames)
else:
    print("No missing frames found.")


No missing frames found.


In [ ]:
# frames to video
import cv2
import os
from pathlib import Path

def render(MAIL, OUT, INPATH, START, END, SUFIX=None):
    # Path to the folder containing frames
    frames_folder = f'/content/drive/MyDrive/BLENDER/{MAIL}/{INPATH}'
    parent_path = Path(frames_folder).parent
    print("Parent path:", parent_path)

    if not SUFIX:
        SUFIX = "v1"

    output_folder = os.path.join(parent_path, OUT)
    print('Output folder ===', output_folder)

    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get the list of frame filenames within the specified range
    frame_files = [
        f'{i:04d}.png'
        for i in range(START, END + 1)
        if os.path.isfile(os.path.join(frames_folder, f'{i:04d}.png'))
    ]

    if not frame_files:
        print("⚠️ No frames found in the specified range.")
        return

    print("Sample frames:", frame_files[:5])

    # Read the first frame to get dimensions
    first_frame_path = os.path.join(frames_folder, frame_files[0])
    frame = cv2.imread(first_frame_path)
    if frame is None:
        print(f"❌ Error: Could not read frame {frame_files[0]}")
        return

    height, width, _ = frame.shape

    # Define output video path
    output_video = os.path.join(output_folder, f'{START}-{END}.{SUFIX}.mp4')

    # Video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video, fourcc, 24, (width, height))

    for frame_file in frame_files:
        frame_path = os.path.join(frames_folder, frame_file)
        frame = cv2.imread(frame_path)
        if frame is not None:
            video_writer.write(frame)
        else:
            print(f"⚠️ Skipping unreadable frame: {frame_file}")

    video_writer.release()
    print("✅ Video conversion complete at:", output_video)

# Example usage (just add your MAIL variable before calling)
# MAIL = "your_email_folder"
render(MAIL, "FINALS", "BLACKOUT/RESULT", 1, 700 )


# **SIMULATION**

In [ ]:
!cp -r "/content/drive/MyDrive/BLENDER/example@GMAIL.COM/SPEED/SPEED.blend" "/content/"

In [ ]:

#Install Blender from google drive to colab
!wget https://download.blender.org/release/Blender4.3/blender-4.3.0-linux-x64.tar.xz
!tar xf blender-4.3.0-linux-x64.tar.xz

# #Connect Google Drive to colab
from google.colab import drive
drive.mount('/content/drive')


#Execute the python script on colab
! ./blender-4.3.0-linux-x64/blender -b "/content/drive/MyDrive/BLENDER/example@GMAIL.COM/SPEED/SPEED.blend" -P "/content/FILE.py"


In [ ]:
#py
import bpy

bpy.ops.fluid.bake_all()

bpy.ops.wm.save_mainfile()

# bake all

In [ ]:
#Connect Google Drive to colab
# from google.colab import drive
# drive.mount('/content/drive')

# #Install Blender from google drive to colab
# !tar xf '/content/drive/My Drive/Colab-Bake/blender-4.0.2-linux-x64.tar.xz' -C "/content"

#Execute the python script on colab
! ./blender-4.3.0-linux-x64/blender -b $filename -P "/content/FILE.py"


In [ ]:
#py
import bpy

bpy.ops.ptcache.free_bake_all()
bpy.ops.ptcache.bake_all()

bpy.ops.wm.save_mainfile()

# **openEXR**

In [ ]:
!wget https://download.blender.org/release/Blender3.3/blender-3.3.8-linux-x64.tar.xz
!tar -xf blender-3.3.8-linux-x64.tar.xz


--2025-02-19 04:51:15--  https://download.blender.org/release/Blender3.3/blender-3.3.8-linux-x64.tar.xz
Resolving download.blender.org (download.blender.org)... 104.22.65.163, 172.67.14.163, 104.22.64.163, ...
Connecting to download.blender.org (download.blender.org)|104.22.65.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 207424760 (198M) [application/octet-stream]
Saving to: ‘blender-3.3.8-linux-x64.tar.xz’

blender-3.3.8-linux 100%[===================>] 197.82M  61.9MB/s    in 3.8s    

2025-02-19 04:51:20 (51.8 MB/s) - ‘blender-3.3.8-linux-x64.tar.xz’ saved [207424760/207424760]



In [ ]:
filename = '/content/drive/MyDrive/BLENDER/example@GMAIL.COM/APPLEPAD/PAD.blend'
OUTPUT = '/content/drive/MyDrive/BLENDER/example@GMAIL.COM/APPLEPAD/####'

In [ ]:
filename = '/content/drive/MyDrive/BLENDER/example@GMAIL.COM/APPLEPAD/PAD.blend'
OUTPUT = '/content/drive/MyDrive/BLENDER/example@GMAIL.COM/APPLEPAD/3.3.8/####'
!./blender-3.3.8-linux-x64/blender -b $filename -noaudio -E 'CYCLES' -o $OUTPUT -s 1 -e 4 -a -F 'PNG' -- --cycles-device CUDA
filename = '/content/drive/MyDrive/BLENDER/example@GMAIL.COM/APPLEPAD/PAD.blend'
OUTPUT = '/content/drive/MyDrive/BLENDER/example@GMAIL.COM/APPLEPAD/4.3.0/####'
!./blender-4.3.0-linux-x64/blender -b $filename -noaudio -E 'CYCLES' -o $OUTPUT -s 1 -e 4 -a -F 'PNG' -- --cycles-device CUDA


# **MOTION BLUR**

In [ ]:
!apt-get -y install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [ ]:
!cp -r '/content/drive/MyDrive/BLENDER/example@GMAIL.COM/SPEED/RESULT4.1' 'content/'

In [ ]:
%cd /content/content/

!ffmpeg -framerate 24 -i %04d.png -filter_complex "tblend=all_mode=average,tblend=all_mode=average,tblend=all_mode=average" -c:v libx264 -pix_fmt yuv420p /content/blurred_output.mp4
yy

/content/content
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --ena

# **ADDON**

In [ ]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Download and extract Blender
!wget https://download.blender.org/release/Blender4.4/blender-4.4.0-linux-x64.tar.xz
!tar xf blender-4.4.0-linux-x64.tar.xz

# Clone the camera_shakify addon from GitHub
!git clone https://github.com/EatTheFuture/camera_shakify.git

# Paths
MAIL = 'example@GMAIL.COM'
addon_dir = '/content/camera_shakify'
addon_module = 'camera_shakify'
filename = f'/content/drive/MyDrive/BLENDER/{MAIL}/BLACKOUT/BLACKOUT_4.4.blend'
OUTPUT = f'/content/drive/MyDrive/BLENDER/{MAIL}/BLACKOUT/RESULT/####'


In [ ]:

# Create script to install the addon
with open("install_addon.py", "w") as f:
    f.write(f"""
import bpy
import sys

# Install and enable the addon from directory
bpy.ops.preferences.addon_install(filepath="{addon_dir}/__init__.py", overwrite=True)
bpy.ops.preferences.addon_enable(module="{addon_module}")
bpy.ops.wm.save_userpref()
print("camera_shakify installed and enabled.")
""")

# Run Blender to install and enable the addon
!./blender-4.4.0-linux-x64/blender -b -P install_addon.py

# Render the scene
!./blender-4.4.0-linux-x64/blender -b {filename} -noaudio -E 'CYCLES' -S 'Scene' -o {OUTPUT} -s 100 -e 100 -a -F 'PNG' -- --cycles-device CUDA
